In [1]:
import sys
import time
import os
import subprocess
import math
import matplotlib.pyplot as plt
import numpy as np
from astropy.table import Table, Column 
from scipy.stats import linregress
from scipy import interpolate
from scipy import polyval, polyfit
from scipy.optimize import curve_fit
from scipy import odr
import pylab as py
from matplotlib import gridspec
import sklearn.datasets as ds
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_regression
import corner
import emcee
import scipy.optimize as op
from scipy.linalg import cholesky, inv,det
from scipy.optimize import minimize
import random
from astropy.table import Table, Column
import pandas as pd
import numpy as np
from datetime import datetime
import time

In [2]:
df = pd.read_csv('test.csv', delimiter='|')
df = df.rename(columns=lambda x: x.strip())
df = df.set_index('PGC')

In [3]:
for col in df.columns:
    print(col)

RA
DE
Glon
Glat
SGL
SGB
Vhel
Vls
D25
b/a
pa
T
Ty
SS
A100
QAs
QAw
ebv
u_mag
g_mag
r_mag
i_mag
z_mag
u_Rasy
g_Rasy
r_Rasy
i_Rasy
z_Rasy
A_u
A_g
A_r
A_i
A_z
Sb/a
Spa
mu0_u
mu0_g
mu0_r
mu0_i
mu0_z
mu50_u
mu50_g
mu50_r
mu50_i
mu50_z
mu90_u
mu90_g
mu90_r
mu90_i
mu90_z
m255_u
m255_g
m255_r
m255_i
m255_z
disk_mu0_u
disk_mu0_g
disk_mu0_r
disk_mu0_i
disk_mu0_z
disk_h_u
disk_h_g
disk_h_r
disk_h_i
disk_h_z
R50_u
R50_g
R50_r
R50_i
R50_z
R90_u
R90_g
R90_r
R90_i
R90_z
R255_u
R255_g
R255_r
R255_i
R255_z
C82_u
C82_g
C82_r
C82_i
C82_z
d_m_ext_u
d_m_ext_g
d_m_ext_r
d_m_ext_i
d_m_ext_z
w1_mag
w2_mag
w1_Rasy
w2_Rasy
A_w1
A_w2
Wb/a
Wpa
mu0_w1
mu0_w2
mu50_w1
mu50_w2
mu90_w1
mu90_w2
m255_w1
m255_w2
disk_mu0_w1
disk_mu0_w2
disk_h_w1
disk_h_w2
R50_w1
R50_w2
R90_w1
R90_w2
R255_w1
R255_w2
C82_w1
C82_w2
d_m_ext_w1
d_m_ext_w2
Qs
Qw
D
L
P
F
N
C
O
V
M
B
U
Note
inc
inc_e
inc_flag
inc_n
Note_inc


In [4]:
bands = ['u', 'g', 'r', 'i', 'z', 'w1', 'w2']

cols = []
cols += [band+'_mag' for band in bands]
cols += [band+'_Rasy' for band in bands]
cols += ['A_'+band for band in bands]
cols += [band+'_Rasy' for band in bands]
cols += ['mu0_'+band for band in bands]
cols += ['mu50_'+band for band in bands]
cols += ['mu90_'+band for band in bands]
cols += ['m255_'+band for band in bands]
cols += ['disk_mu0_'+band for band in bands]
cols += ['disk_h_'+band for band in bands]
cols += ['R50_'+band for band in bands]
cols += ['R90_'+band for band in bands]
cols += ['R255_'+band for band in bands]
cols += ['C82_'+band for band in bands]
cols += ['d_m_ext_'+band for band in bands]
cols += ['Wb/a', "Wpa", "Sb/a", "Spa"]

for col in cols:
    df[col] = df[col].apply(lambda x: x if x!=0 else np.nan)
    

cols = []
cols += ['disk_mu0_'+band for band in bands] 

for col in cols:
    df[col] = df[col].apply(lambda x: x if x!=-9.99 else np.nan)

In [5]:
df["Sb/a"]

PGC
2           NaN
4          0.24
12         0.22
16         0.52
18          NaN
           ... 
6719913     NaN
6724044     NaN
6724069    0.81
6724508    0.55
6726345     NaN
Name: Sb/a, Length: 19905, dtype: float64

In [6]:
df[df.columns[:50]].head()

,RA,DE,Glon,Glat,SGL,SGB,Vhel,Vls,D25,b/a,...,mu50_u,mu50_g,mu50_r,mu50_i,mu50_z,mu90_u,mu90_g,mu90_r,mu90_i,mu90_z
PGC,,,,,,,,,,,,,,,,,,,,,
2,0.0070,47.2745,113.9553,-14.6992,341.6440,20.7388,5017.0,5310.0,1.86,0.63,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0.0146,23.0876,107.8322,-38.2729,316.0587,18.4514,4458.0,4706.0,0.85,0.22,...,24.21,22.88,22.25,22.08,22.12,26.18,24.52,24.13,24.35,24.48
12,0.0358,-6.3739,90.1920,-65.9300,286.4249,11.3511,6531.0,6668.0,1.41,0.22,...,26.75,23.99,28.32,NaN,23.57,26.09,24.99,27.32,NaN,23.47
16,0.0471,-5.1588,91.6005,-64.8656,287.6119,11.7030,5713.0,5855.0,0.93,0.49,...,23.12,21.78,21.05,20.72,20.47,25.73,23.61,22.87,22.66,22.69
18,0.0540,46.9651,113.9223,-15.0086,341.3139,20.7013,5366.0,5658.0,0.95,0.72,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
len(df)

19905

In [8]:
df.to_csv("test_bar", sep='|')